In [69]:
import logging #code adapted from https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import os

In [82]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) #remove weird characters
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwords
    return text

In [93]:
script_dir = os.path.abspath('')
raw_speeches = open(script_dir + "\\hein-bound\\hein-bound\\speeches_043.txt")
raw_map = open(script_dir + "\\hein-bound\\hein-bound\\043_SpeakerMap.txt")
raw_speeches = raw_speeches.readlines()
raw_map = raw_map.readlines()
raw_map = [x.split('|') for x in raw_map[1:]]
speech = [x[10:] for x in raw_speeches[1:]]
id = [x[:9] for x in raw_speeches[1:]]
speeches = []
for m in raw_map:
    i = (int(m[1])%1000000)-1
    speeches.append({'speech': clean_text(speech[i]), 'party': m[7]})



In [95]:
X = [x['speech'] for x in speeches]
y = [x['party'] for x in speeches]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)



In [100]:
print(X_train[:5])
print(y_train[:5])

['possible way escape plain meaning opinion course friend mean escape', 'also intrbduced bill 1 r 895 fol relief reuben j stewart fayette county alabama read first second time referred committee claims ordered printed', 'move amend paragraph desire ask question gentleman bill charge beginning paragraph find words conversion rifling heavy guns 75000 niderstand giens converted smoothbores rifles hence word riling surperflnons unnecessary would suggest place word rifling 4ho word tests read conversion tests heavy gunas word irifling mean anything word 11 convert 1 conversion covers whole grond', 'well chair decide', 'permanent appropriation']
['R', 'R', 'R', 'D', 'R']


In [103]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)


print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.7296467772980284


In [98]:
# try chaning some logisitic regression parameters?
# Doesn't seem to converge for some reason
logreg = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', LogisticRegression(n_jobs=1, C=1e5, max_iter=1000))])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
#print(classification_report(y_test, y_pred,target_names=my_tags))

C:\Users\rdp81\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
UsageError: Line magic function `%%time` not found.
